In [59]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings

warnings.filterwarnings('ignore')

pd.options.display.max_columns = None
pd.set_option('float_format', '{:.2f}'.format)

In [60]:
current_date = dt.date.today()
day = current_date.day
month = current_date.month
year = current_date.year

way = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Илларионов\Выгрузка Еком\\'

In [61]:
data_IM = pd.read_excel(rf'{way}\{month}.{year}_продажи еком.xlsx')

In [62]:
# смотрим продажи с последнего мониторинга
data = dt.datetime.isoweekday(dt.date.today())
if data == 1:
    last_monitoring = (dt.date.today() - dt.timedelta(days=3)).strftime('%Y-%m-%d')
else:
    last_monitoring = (dt.date.today() - dt.timedelta(days=2)).strftime('%Y-%m-%d')
    
data_IM = data_IM[data_IM['created'] >= f'{last_monitoring}']

In [63]:
data_IM.sample(5)

,id,created,name,state_order,recipient_phone,recipient_email,state_item,delivery_type,payment_type,employee_id,type_ibox,type_name,tov,kbk_recieve_delivery,avg_weight,avg_net_weight,materials,proba,design,type1,type3,tg,brand,vendor_article,number,price,client_type,date_status_order_done,date_status_item_returned,product_id,cost_price,shk,kbk_collect_delivery,manzana,percent_by_sku,manzana_for_item,price_wo_manzana
881,ea523c03-55d0-4245-9dc3-0657adba1547,2024-06-02 07:37:43.323,Золотое кольцо с фианитами,done,79053442889.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ1265214,2-18-014,1.14,1.10,NaN,585.00,ИФ ДОРОЖКА,КОЛЬЦО,КРАСНЫЙ,ИФ КОЛЬЦА,NaN,20-02-0001-80362,2406020737607,7000.00,MOBILE,2024-06-02 11:00:19.804,NaN,1048003,2471.34,2078604055655,2-18-014,2916,0.48,1400.00,5600.00
93,15215b92-1764-4d79-b65e-051ec9e76227,2024-05-31 11:00:59.600,Золотые серьги с фианитами,done,79141471611.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_SBP,Тинькофф СБП,ТОВ1271820,3-09-032,3.25,3.16,NaN,375.00,ИФ СЕР С ПОДВЕСКАМИ,СЕРЬГИ,КРАСНЫЙ,ИФ СЕРЬГИ,NaN,14-92-0001-25266#375,2405311100839,19969.95,MOBILE,2024-06-01 02:45:11.819,NaN,1785987,7470.07,2078610368996,3-09-032,3993,1.00,3993.00,15976.95
112,1aac0143-1bce-4e8d-91f6-b37eee605c96,2024-05-31 22:04:28.602,Серебряная подвеска Дракон с чернением,done,79139714318.00,NaN,done,PICKUP_NOW,CASHIER,NaN,CASHIER,[CASHIER] CASHIER - CASHIER,ТОВ1302374,4-05-058,2.85,2.85,NaN,925.00,СИ ПРОЧЕЕ,ПОДВЕС ДЕКОРАТИВНЫЙ,ЧЁРНЫЙ,СИ ПОДВЕС БК,NaN,50-76-0000-15126,2405312204684,2180.00,MOBILE,2024-06-02 08:13:24.347,NaN,1336258,550.47,2078607416283,4-05-058,0,0.00,0.00,2180.00
510,86126a82-8bf2-4015-b8a4-9e59006aee3a,2024-06-01 03:10:55.616,Золотые серьги,done,79233836116.00,NaN,done,PICKUP_NOW,CASHIER,NaN,CASHIER,[CASHIER] CASHIER - CASHIER,ТОВ1168321,3-03-020,2.63,2.63,NaN,375.00,ИФ ШТАМП П-ПЛАСТИНА,СЕРЬГИ,КРАСНЫЙ,БК СЕРЬГИ,NaN,18-93-0000-28314#375,2406010310262,11337.50,MOBILE,2024-06-01 07:48:59.442,NaN,2220888,5885.48,2078607819011,3-03-020,0,0.00,0.00,11337.50
800,d2ec093b-6c18-40b2-895b-51333b5d32fd,2024-06-01 07:03:21.520,Золотая пустотелая цепочка Бисмарк с алмазной ...,done,NaN,NaN,done,PICKUP_NOW,CASHIER,869cc2e4-ca09-4532-a4fb-d3f53f3e0942,SBER,[CASHIER] SBER - QR,ТОВ0992906,6-04-082,6.93,6.96,NaN,585.00,БИСМАРК ПУСТ,ЦЕПЬ,КРАСНЫЙ,ЦБ ЦЕПИ,NaN,"НЦ12-316ПГ D0,70",2406010703222,49240.00,RETAIL,2024-06-01 07:05:04.038,NaN,1267605,25718.31,2078604783522,6-04-082,0,0.00,0.00,49240.00


In [64]:
data_IM = data_IM.drop_duplicates()
data_IM = data_IM.reset_index(drop=True)

data_IM = data_IM[(data_IM['state_order'] != 'returned') & (data_IM['state_order'] != 'canceled')]
data_IM = data_IM[~((data_IM['delivery_type'] == 'PICKUP_NOW') & (data_IM['payment_type'] == 'CASHIER') & (data_IM['type_ibox'] == 'CASHIER'))|
                ((data_IM['delivery_type'] == 'PICKUP') & (data_IM['payment_type'] == 'CASHIER') & (data_IM['type_ibox'] == 'CASHIER'))]

In [65]:
data_IM.sample(5)

,id,created,name,state_order,recipient_phone,recipient_email,state_item,delivery_type,payment_type,employee_id,type_ibox,type_name,tov,kbk_recieve_delivery,avg_weight,avg_net_weight,materials,proba,design,type1,type3,tg,brand,vendor_article,number,price,client_type,date_status_order_done,date_status_item_returned,product_id,cost_price,shk,kbk_collect_delivery,manzana,percent_by_sku,manzana_for_item,price_wo_manzana
738,ec0a6a3f-a4a0-4dd4-9085-8c1cd9afe09c,2024-06-02 08:34:03.097,Кольцо из золота с фианитами,done,79600095588.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - APP,ТОВ1030665,1-02-003,2.31,2.06,NaN,585.00,ИФ С КРУПНЫМ КАМНЕМ,КОЛЬЦО,КРАСНЫЙ,ОПТ ИФ,NaN,КСК 717#ТД,2406020834399,22945.00,MOBILE,2024-06-02 08:49:38.914,NaN,3632216,5193.77,2078601077281,1-02-003,3807,1.00,3807.00,19138.00
595,be281f51-9059-4069-a9d8-b9d47fdfd5cd,2024-06-01 14:15:45.453,Золотые серьги Сердце с фианитами,done,79644216241.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,YANDEX_PAY,Яндекс пэй для мп,ТОВ1070715,3-09-050,0.78,0.78,NaN,585.00,ИФ ДЕТСКИЕ Б/Ц,СЕРЬГИ,КРАСНЫЙ,ИФ СЕРЬГИ,NaN,13-02-0001-26498,2406011415184,5490.00,MOBILE,2024-06-02 07:24:03.968,NaN,4811693,2933.20,2078610481895,3-09-050,0,0.00,0.00,5490.00
618,c60c9383-edbd-48f0-9bdd-41152b4c8993,2024-06-02 14:01:10.713,Золотое плоское обручальное кольцо 5 мм,done,79870964301.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,00dacdee-27bb-472e-96c5-c20f97598faf,SBER,[YANDEX_KASSA] SBER - URL,ТОВ0887594,4-06-019,3.60,3.60,NaN,585.00,ИФ ОБРУЧ ПЛОСКАЯ,КОЛЬЦО ОБРУЧАЛЬНОЕ,КРАСНЫЙ,БК КОЛЬЦА ОБРУЧ,NaN,КО 15-00,2406021401158,22320.00,MOBILE,2024-06-02 14:04:46.774,NaN,2004212,12233.80,2078610872644,4-06-019,0,0.00,0.00,22320.00
668,d617133f-4884-4d91-856c-08e9a30b18af,2024-06-02 05:56:43.519,Серебряные серьги с янтарем и позолотой,done,79255331879.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ1291504,2-09-160,1.73,1.53,NaN,925.00,СИ ЯНТАРЬ,СЕРЬГИ,ЖЕЛТЫЙ,СИ ЯНТАРЬ,NaN,122042202ga,2406020556311,1700.00,MOBILE,2024-06-02 18:02:31.877,NaN,1686933,555.26,2078605979099,2-09-160,340,1.00,340.00,1360.00
429,89de6761-174e-4861-91f7-8e92ef6c35cf,2024-05-31 15:14:58.458,Золотое обручальное кольцо с алмазной гранью 5 мм,done,79283502820.00,777lina77790@mail.ru,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ1008404,6-05-065,2.62,2.62,NaN,585.00,ИФ ОБРУЧ АЛМ ГР ЛЮКС,КОЛЬЦО ОБРУЧАЛЬНОЕ,КРАСНЫЙ,БК КОЛЬЦА ОБРУЧ,NaN,14-2065-10-00,2405311514193,19331.00,MOBILE,2024-06-01 13:09:03.628,NaN,3540657,6142.71,2078520109921,6-05-065,3866,1.00,3866.00,15465.00


In [66]:
data_IM['date_status_order_done'] = pd.to_datetime(data_IM['date_status_order_done'], format = '%Y-%m-%d')

In [67]:
def str_float(row):
    row = str(row)
    row = row.replace(',','.')
    return row

data_IM['price'] = data_IM['price'].apply(str_float)
data_IM['price'] = data_IM['price'].astype(float)

data_IM['avg_net_weight'] = data_IM['avg_net_weight'].apply(str_float)
data_IM['avg_net_weight'] = data_IM['avg_net_weight'].astype(float)

In [68]:
data_IM = data_IM[data_IM['price'] > 1].reset_index(drop=True)
data_IM = data_IM[data_IM['avg_net_weight'] > 0].reset_index(drop=True)

In [69]:
data_IM['Товарное направление'] = np.where(data_IM['tg'].isin(['БК ПОДВЕС КУЛЬТ', 'БК СЕРЬГИ', 'БК КОЛЬЦА','БК КОЛЬЦА ОБРУЧ','БК ПОДВЕС ДЕКОР',  'БК ПЕЧАТКИ','БК АКСЕССУАРЫ','ОПТ БК']), 'БК',
                                np.where(data_IM['tg'].isin(['ПДК СЕРЬГИ', 'ПДК КОЛЬЦА','ПДК ПОДВЕСКИ']), 'ПДК',
                                np.where(data_IM['tg'].isin(['ДК СЕРЬГИ',  'ДК КОЛЬЦА',    'ДК ПОДВЕСКИ',    'ДК КОЛЬЦА ОБРУЧ']), 'ДК',
                                np.where(data_IM['tg'].isin(['ИФ СЕРЬГИ',  'ИФ КОЛЬЦА', 'ИФ ПОДВЕС ДЕКОР', 'ИФ ПЕЧАТКИ', 'ИФ ПОДВЕС КУЛЬТ','ИФ КОЛЬЦА ОБРУЧ','ИФ АКСЕССУАРЫ']), 'ИФ',
                                np.where(data_IM['tg'].isin(['ЦБ ЦЕПИ', 'ЦБ БРАСЛЕТЫ']), 'ЦБ',
                                np.where(data_IM['tg'].isin(['СИ ПОДВЕС БК','СИ КОЛЬЦО ИФ','СИ БРАСЛЕТ БК','СИ КОЛЬЦО БК','СИ ЦЕПЬ','СИ СЕРЬГИ БК','СИ СЕРЬГИ ИФ','СИ ПОДВЕС ИФ','СИ СЕРЬГИ РАЗН','СИ СЕРЬГИ ЦВ/К',\
                                                            'СИ КОЛЬЦО ЦВ/К','','СИ ШАРМЫ','СИ ОБРУЧАЛЬНОЕ','СИ КОЛЬЦО ПЕЧАТ','СИ ПОДВЕСЦВ/К','СИ БРАСЛЕТ СК','СИ ЖЕМЧУГ',\
                                                            'СИ АКСЕССУАР','СИ ДФРС ПРОЕКТ','СИ КЕРАМИКА','СИ ПОСУДА','СИ ЯНТАРЬ','СИ ЭМАЛЬ','СИ ВЫРИЦА']), 'СИ',
                                np.where(data_IM['tg'].isin(['ЧАСЫ']), 'ЧАСЫ','другое')))))))

data_IM['Товарное направление'].value_counts()

Товарное направление
БК        153
СИ        137
ИФ        111
ЦБ         98
ДК         72
ПДК        42
другое     23
Name: count, dtype: int64

In [70]:
proverka = data_IM[data_IM['Товарное направление']=='другое']
proverka['tg'].unique()

array(['ОПТ ИФ', 'СИ СЕРЬГИ\xa0 ЦВ/К', 'СИ КОЛЬЕ СК', 'ПДК ПЕЧАТКИ',
       'СИ ПОДВЕС\xa0ЦВ/К', 'СИ БРАСЛЕТ БК Декор', 'ВЫРИЦА СЕРЕБРО',
       'ДК ПЕЧАТКИ', 'СИ КОЛЬЕ ЦВ/К'], dtype=object)

In [71]:
data_IM = data_IM[~(data_IM['name'].str.contains('каучук')
        |data_IM['name'].str.contains('шнурок')
        |data_IM['name'].str.contains('текстиль')
        |data_IM['name'].str.contains('хлопок')
        |data_IM['name'].str.contains('шелк')
        |data_IM['name'].str.contains('на нитке')
        |data_IM['name'].str.contains('кожа')
        |data_IM['name'].str.contains('жемчуг'))]

In [72]:
data_IM.info()

<class 'pandas.core.frame.DataFrame'>
Index: 625 entries, 0 to 635
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         625 non-null    object        
 1   created                    625 non-null    datetime64[ns]
 2   name                       625 non-null    object        
 3   state_order                625 non-null    object        
 4   recipient_phone            623 non-null    float64       
 5   recipient_email            205 non-null    object        
 6   state_item                 625 non-null    object        
 7   delivery_type              625 non-null    object        
 8   payment_type               625 non-null    object        
 9   employee_id                41 non-null     object        
 10  type_ibox                  625 non-null    object        
 11  type_name                  625 non-null    object        
 12  tov          

In [73]:
data_IM = data_IM[(data_IM['Товарное направление'].isin(['БК', 'ЦБ']))&((data_IM['proba'] == '585')|(data_IM['proba'] == 585.00))]

In [74]:
data_IM['Товарное направление'].value_counts()

Товарное направление
БК    117
ЦБ     78
Name: count, dtype: int64

In [75]:
data_IM['proba'].unique()

array([585.])

In [76]:
data_IM['date_status_order_done'].value_counts()

date_status_order_done
2024-06-01 07:36:19.962    1
2024-06-02 12:26:19.319    1
2024-06-01 09:49:10.419    1
2024-06-02 12:36:14.362    1
2024-06-01 04:55:20.014    1
                          ..
2024-06-01 12:22:35.789    1
2024-06-01 10:51:33.182    1
2024-06-01 07:25:12.690    1
2024-06-02 12:23:33.736    1
2024-06-01 10:52:46.129    1
Name: count, Length: 195, dtype: int64

In [77]:
previous_date = data_IM['date_status_order_done'].min()

In [78]:
filtr_date = data_IM['date_status_order_done'].max()
print(filtr_date)

2024-06-02 18:03:31.567000


In [79]:
data_IM['date_status_order_done'] = pd.to_datetime(data_IM['date_status_order_done'], format='%Y-%m-%d')

In [80]:
data_IM['date_status_order_done'] = pd.to_datetime(data_IM['date_status_order_done'].apply(lambda x: x.strftime("%Y-%m-%d")))

In [81]:
data_today = data_IM.query("date_status_order_done >= @previous_date")
data_today.head()

,id,created,name,state_order,recipient_phone,recipient_email,state_item,delivery_type,payment_type,employee_id,type_ibox,type_name,tov,kbk_recieve_delivery,avg_weight,avg_net_weight,materials,proba,design,type1,type3,tg,brand,vendor_article,number,price,client_type,date_status_order_done,date_status_item_returned,product_id,cost_price,shk,kbk_collect_delivery,manzana,percent_by_sku,manzana_for_item,price_wo_manzana,Товарное направление
9,01ed8497-e2e0-4271-8712-9ccc1637593a,2024-06-02 08:20:50.844,Золотые серьги,done,79952735042.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ1262625,3-06-006,0.72,0.72,NaN,585.00,ИФ ПРОДЕВКИ,СЕРЬГИ,КРАСНЫЙ,БК СЕРЬГИ,NaN,19-01-0000-11248,2406020820922,7215.00,MOBILE,2024-06-02,NaN,1208928,2631.15,2078610813763,3-06-006,1443,1.00,1443.00,5772.00,БК
13,050cc207-9421-4b7c-817c-74141546ccf6,2024-06-02 16:05:55.295,Золотой крест,done,79176064960.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_SBP,Тинькофф СБП,ТОВ1165005,5-02-017,0.29,0.29,NaN,585.00,ИФ КРЕСТ,ПОДВЕС КУЛЬТОВЫЙ,КРАСНЫЙ,БК ПОДВЕС КУЛЬТ,NaN,35-0479-11-00,2406021605253,2834.00,MOBILE,2024-06-02,NaN,1747481,1003.74,2078610790281,5-02-017,566,1.00,566.00,2268.00,БК
17,05d1c173-4357-45c1-8c66-929fafb1c44c,2024-06-01 19:23:42.075,Золотой полновесный браслет Лав с алмазной гранью,done,79043500959.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_PAY,Тинькофф Долями,ТОВ01261564,1-03-004,1.47,1.47,NaN,585.00,ЛАВ,БРАСЛЕТ,КРАСНЫЙ,ЦБ БРАСЛЕТЫ,NaN,40-01-0035-30087,2406011923144,16160.00,MOBILE,2024-06-02,NaN,4648635,3674.37,2078601829980,1-03-004,0,0.00,0.00,16160.00,ЦБ
25,0a8e4b07-2735-4b01-a267-51d69c46fe26,2024-05-31 02:34:39.119,Золотое гладкое обручальное кольцо 3 мм,done,79041867633.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_PAY,Тинькофф Долями,ТОВ0832947,5-02-061,1.64,1.64,NaN,585.00,ИФ ОБРУЧ ГЛАДКАЯ,КОЛЬЦО ОБРУЧАЛЬНОЕ,КРАСНЫЙ,БК КОЛЬЦА ОБРУЧ,NaN,КО 03-00#ТД,2405310234843,13182.00,MOBILE,2024-06-02,NaN,2002928,4251.07,2078600551148,5-02-061,0,0.00,0.00,13182.00,БК
26,0a8e4b07-2735-4b01-a267-51d69c46fe26,2024-05-31 02:34:39.119,Золотое гладкое обручальное кольцо 4 мм,done,79041867633.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_PAY,Тинькофф Долями,ТОВ0832969,5-02-061,2.51,2.51,NaN,585.00,ИФ ОБРУЧ ГЛАДКАЯ,КОЛЬЦО ОБРУЧАЛЬНОЕ,КРАСНЫЙ,БК КОЛЬЦА ОБРУЧ,NaN,КО 04-00#ТД,2405310234843,12890.00,MOBILE,2024-06-02,NaN,2002944,9009.67,2078609859800,5-02-061,0,0.00,0.00,12890.00,БК


In [82]:
data_today['date_status_order_done'].unique()

<DatetimeArray>
['2024-06-02 00:00:00']
Length: 1, dtype: datetime64[ns]

In [83]:
data_today.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101 entries, 9 to 629
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         101 non-null    object        
 1   created                    101 non-null    datetime64[ns]
 2   name                       101 non-null    object        
 3   state_order                101 non-null    object        
 4   recipient_phone            101 non-null    float64       
 5   recipient_email            24 non-null     object        
 6   state_item                 101 non-null    object        
 7   delivery_type              101 non-null    object        
 8   payment_type               101 non-null    object        
 9   employee_id                14 non-null     object        
 10  type_ibox                  101 non-null    object        
 11  type_name                  101 non-null    object        
 12  tov          

In [84]:
data_today['цена за гр'] = data_today['price_wo_manzana'] / data_today['avg_weight']
data_today['цена за гр'].describe()

count     101.00
mean     6392.46
std      1490.22
min      4171.52
25%      5218.62
50%      6200.00
75%      7187.05
max     14489.51
Name: цена за гр, dtype: float64

In [85]:
data_today['avg_weight'].describe()

count   101.00
mean      2.15
std       1.22
min       0.28
25%       1.13
50%       2.21
75%       2.72
max       5.84
Name: avg_weight, dtype: float64

In [86]:
data_today = data_today[~data_today['tg'].str.contains('ОПТ БК|БК ПЕЧАТКИ|ОПТ КОНГО')]

In [87]:
data_today = data_today[~data_today['client_type'].str.contains('OZON')]

In [88]:
print('Средняя цена продажи еком: ', (data_today['price_wo_manzana'].sum()/data_today['avg_weight'].sum()).round(0))

Средняя цена продажи еком:  6069.0


In [89]:
date_now = dt.datetime.now().date().strftime('%d.%m.%Y')

In [90]:
data_today.to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\ecom-sales\online_sales_{date_now}.xlsx', index=False)

In [91]:
tn = data_today.groupby('tg', as_index=False)[['price_wo_manzana', 'avg_weight', 'cost_price']].agg('sum')
tn['price_gramm'] = (tn['price_wo_manzana'] / tn['avg_weight']).round(0)

In [92]:
tn['price_gramm'] = tn['price_gramm'].astype('int')

In [93]:
tn['Наценка продажи онлайн'] =  tn['price_wo_manzana'] / tn['cost_price'] - 1

In [94]:
tn

,tg,price_wo_manzana,avg_weight,cost_price,price_gramm,Наценка продажи онлайн
0,БК КОЛЬЦА,24539.00,4.59,9773.72,5346,1.51
1,БК КОЛЬЦА ОБРУЧ,544539.00,102.81,302157.60,5297,0.80
2,БК ПОДВЕС ДЕКОР,28208.74,3.95,12208.52,7141,1.31
3,БК ПОДВЕС КУЛЬТ,89360.98,12.36,36834.01,7230,1.43
4,БК СЕРЬГИ,5772.00,0.72,2631.15,8017,1.19
5,ЦБ БРАСЛЕТЫ,247072.00,35.29,109409.43,7001,1.26
6,ЦБ ЦЕПИ,380012.83,57.68,174822.91,6588,1.17


In [95]:
print('Средняя цена продажи еком: ', (tn['price_wo_manzana'].sum() / tn['avg_weight'].sum()).round(0))

Средняя цена продажи еком:  6069.0


In [96]:
tn.to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\ecom-sales\ecom_sales_tg\\ecom_sales_{date_now}.xlsx', index=False)

In [97]:
tn = tn.drop(columns=['price_wo_manzana', 'avg_weight', 'cost_price', 'Наценка продажи онлайн'])

In [98]:
tn.columns = ['tg', '585*Золотой онлайн продажи']

In [99]:
day

3

In [100]:
pd.read_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sklv_sl_data\grouped_data\result_{date_now}.xlsx').merge(tn, how = 'left', on = 'tg').to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sklv_sl_data\grouped_data\result_{date_now}.xlsx', index=False)